# Lecture 6 Hands-on F24 - ARIMA forecasting

The purpose of this hands-on is to understand how to fit, use and evaluate ARIMA models in Python. Some simple time-series are used as examples because energy-related ones (such as prices or consumption) have more complicated characteristics and require more advanced models. You will work with such time-series in the hands-on of Lecture 7.

You will also familiarize yourselves with important operations such as splitting your datasets into training/testing ones, calculating error metrics, visualizing the forecasts etc.

In [ ]:
# Import libraries

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from IPython.display import Markdown as md
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pmdarima as pm
from math import sqrt
import warnings
import numpy as np

warnings.filterwarnings("ignore")

## Simulate AR processes and fit AR models

- Simulate an AR(2) process and plot the time series
- Split the time series into a training and testing dataset
- Plot the ACF and PACF plot
- You can experiment and create different time series by modifying the AR and MA parameters

In [ ]:
# Define the AR terms in the simulated process
arparams = np.array([0.8, -0.3])
ar = np.r_[1, -arparams]

# Ignore the MA terms for now and keep them equal to 0
maparams = np.array([0])
ma = np.r_[1, maparams]

# Create the model to generate the samples
arma_process = sm.tsa.ArmaProcess(ar, ma)
data = arma_process.generate_sample(500)

# Split between a training and a testing dataset
n = int(0.8*len(data))
N = int(len(data))
train, test = pm.model_selection.train_test_split(data, train_size=n)

# Plot the result
plt.figure(figsize=(8, 4), dpi=100)
plt.plot(np.arange(1,n+1), train)
plt.plot(np.arange(n+1,N+1), test)
plt.legend(["Training set", "Testing set"])
plt.tight_layout()
plt.xlabel("t")
plt.ylabel("x")
plt.grid(alpha=0.25)
plt.show()

## Plot the ACF/PACF plots

In [ ]:
# Check ACF plot
pm.plot_acf(train, lags=20, title = "ACF", alpha = 0.05);
# Check the PACF plot
pm.plot_pacf(train, method='ywm', lags=20, title = "PACF", alpha = 0.05);

# Note that alpha indicates the used confidence in the correlation plots, with 95% (alpha = 0.05) being the standard one.
# You can experiment with different values of alpha
# Lags controls the number of lagged values in the plots

# Fitting models and observing the summary

You can now try to fit different models and evaluate which is the better option.
- We start by fitting model1 as AR(1) and model2 as AR(2)
- You can use: <span style="color:blue">pm.arima.ARIMA(order=(p, d, q))</span> to fit a model of your own choice by setting p, d, q
- The summary provides valuable information
- You can search for the Ljung-Box (L1) (Q) statistic and why it's different in the two models
- You can also search the AIC or BIC statistics. Which model has lower values?

In [ ]:
model1 = pm.arima.ARIMA(order=(1, 0, 0))
model1.fit(train)
print("Summary of model 1")
print(model1.summary())

model2 = pm.arima.ARIMA(order=(2, 0, 0))
model2.fit(train)
print("\n Summary of model 2")
print(model2.summary())

## Evaluating the fit of models

- We can plot the diagnostics as below
- The diagnostics consist of 4 subplots for the normalized residuals of the training dataset:
  - The time-series of the residuals -> those should look like white noise
  - The histogram should show a normal distribution (closer to thes shown N(0,1))
  - The Q-Q plot of the residuals (they should lie as close as possible to the red line)
  - The auto-correlation plot of the residuals. No significant correlation should be observed
 - Which of the two fulfils all criteria?
 - Does the number of samples play a role? We set N = 500 initially

In [ ]:
model1_diagnostics = model1.plot_diagnostics(lags=20, fig=plt.figure())
plt.tight_layout()
plt.show()

In [ ]:
model2_diagnostics = model2.plot_diagnostics(lags=20, fig=plt.figure())
plt.tight_layout()
plt.show()

## Predicting with ARIMA models

You can use the fitted model to predict future values!

### Rolling forecasts
Here we are at time step $t$ and forecast the next step $t+1$. Next, we observe the realized true value at $t+1$, and based on this we predict for $t+2$ and so on. In other words, we update our model with new observations and produce new forecasts at each step $t$.

Observe the forecast and how it behaves. Discuss it with your colleagues. Experiment with the order of the model.

In [ ]:
# Create a list to store the forecasted values with our model AR(2)
frc_values = []

# Fit the model
model = pm.arima.ARIMA(order=(2, 0, 0))
model.fit(train)

for k in range(len(test)):
    # Predict 1-step ahead
    m = model.predict(1)[0]
    # Append to the frc_values list
    frc_values.append(m)
    # Update the model with the last-seen value
    model.update(test[k])

In [ ]:
# Plot the result
plt.figure(figsize=(8, 4), dpi=100)
plt.plot(range(0,len(train)), train)
plt.plot(range(len(train)+1,len(data)+1), test)
plt.plot(range(len(train)+1,len(data)+1), frc_values)
plt.xlim([len(train)-50,len(data)])
plt.xlabel("t")
plt.xlabel("x")
plt.legend(["Historical data", "Real values", "Forecasted values"])
plt.tight_layout()
plt.show()

## Calculate performance metrics

- Below we calculate the RMSE and MAE
- How else would you assess the performance of the model?
- Do you need to check the residuals?

In [ ]:
rmse_AR = sqrt(mean_squared_error(test, frc_values))
MAE_AR = mean_absolute_error(test, frc_values)

print("The model gives an RMSE of", float("{:.3f}".format(rmse_AR)))
print("The model gives an MAE of", float("{:.3f}".format(MAE_AR)))

## Forecast in one-go

Here we produce forecasts for the whole test dataset of in one-go, i.e., without continuously updating the model with new observations.

In [ ]:
model = pm.arima.ARIMA(order=(2, 0, 0))
model.fit(train)
frc_values_onego = model.predict(len(test))

# Plot the result
plt.figure(figsize=(8, 4), dpi=100)
plt.plot(range(0,len(train)), train)
plt.plot(range(len(train)+1,len(data)+1), test)
plt.plot(range(len(train)+1,len(data)+1), frc_values_onego)
plt.xlim([len(train)-50,len(data)])
plt.xlabel("t")
plt.ylabel("x")
plt.legend(["Historical data", "Real values", "Forecasted values"])

# Calculate errors
rmse_AR = sqrt(mean_squared_error(test, frc_values_onego))
MAE_AR = mean_absolute_error(test, frc_values_onego)
print("The model gives an RMSE of", float("{:.3f}".format(rmse_AR)))
print("The model gives an MAE of", float("{:.3f}".format(MAE_AR)))

### Compare the two forecasts. Why do they behave so differently?

# ARIMA example

Consider the following dataset

In [ ]:
data = [0.643153, 0.565413, 1.02305, -0.295435, -1.63548, 0.906491, 0.151371, 1.58402,
        -1.38134, 0.0901513, -0.436639, 0.288482, 0.327499, 0.607486, 1.32946, -0.295326,
        1.12644, 1.79775, 2.7662, -1.22518, 0.682238, -0.552159, 2.19328, 2.14682, -1.08043,
        2.03189, 0.833014, 1.32183, 0.397554, 3.01139, 3.48769, 2.85506, 1.25216, 1.33211,
        2.97688, 0.986708, 2.63425, 1.06554, 1.03385, 2.93462, 2.65984, 0.72205, 0.288134,
        1.66116, 2.25676, 1.9578, 4.04734, 4.56202, 3.55477, 3.08978, 3.03434, 2.6849, 1.96305,
        3.81255, 2.37639, 3.58423, 3.53872, 3.49335, 3.49195, 3.16102, 4.69291, 4.89491,
        3.75537, 3.70994, 2.60701, 3.21296, 2.62482, 3.69524, 2.97964, 3.59984, 4.10841,
        3.65618, 3.91009, 4.0143, 3.23322, 2.98325, 2.5568, 3.11426, 2.02182, 4.22683,
        3.98335, 3.54927, 5.10158, 5.23455, 5.16627, 4.58551, 4.91136, 4.22619, 4.261, 6.33937,
        6.6991, 3.6015, 5.33029, 5.04643, 4.40196, 3.55238, 3.12268, 5.79854, 3.46685, 5.56311]

### Split into training and testing dataset

In [ ]:
# Split between a training and a testing dataset
n = int(0.8*len(data))
N = int(len(data))
train, test = pm.model_selection.train_test_split(data, train_size=n)

### Plot the dataset
Is this series stationary by observing it? Plot the differenced time-series

In [ ]:
plt.figure(figsize=(8, 4), dpi=100)
plt.plot(train[1:], label = "original")
plt.plot(np.diff(train), label = "differenced")
plt.tight_layout()
plt.legend(["original", "differenced"])
plt.xlabel("t")
plt.ylabel("x")
plt.grid(alpha=0.25)
plt.show()

### Plot the ACF/PACF plots

In [ ]:
# Check ACF plot
pm.plot_acf(np.diff(train), lags=10, title = "ACF", alpha = 0.05);
# Check the PACF plot
pm.plot_pacf(np.diff(train), method='ywm', lags=10, title = "PACF", alpha = 0.05);

### Fit a model

In [ ]:
# Fit the model
model = pm.arima.ARIMA(order=(1, 1, 1))
model.fit(train)

model_diagnostics = model.plot_diagnostics(lags=10, fig=plt.figure())
plt.tight_layout()
plt.show()

### Forecast values

- Now you can assess your model
- Fit a model of your choice and experiment
- Check the metrics and plot any diagnostics you want

In [ ]:
frc_values = []
model = pm.arima.ARIMA(order=(1, 1, 1))
model.fit(train)

for k in range(len(test)):
    m = model.predict(1)[0]
    frc_values.append(m)
    model.update(test[k])

rmse_AR = sqrt(mean_squared_error(test, frc_values))
MAE_AR = mean_absolute_error(test, frc_values)

print("The model gives an RMSE of", float("{:.3f}".format(rmse_AR)))
print("The model gives an MAE of", float("{:.3f}".format(MAE_AR)))

### Using auto-arima

- Fortunately, there is a method that can automatically fit the best model for us
- You can fit a non-seasonal model as: <span style="color:blue">model = pm.auto_arima(train, trace = True, seasonal = False, stepwise=True, maxiter=10)</span>
- Did you choose the same model as the built-in method? Who gets a better performance?

In [ ]:
model = pm.auto_arima(train, trace = True, seasonal = False, stepwise=True, maxiter=10)

# Automating the forecasting process

- Now you are ready to automate the whole process! Follow the steps below and build a function filling the empty parts
- Experiment with the order d and discuss the result in the forecasting
- Check the pmdarima documentation on how to use auto arima with a fixed d

In [ ]:
def Forecaster(data, n, d):
    """
    data: the data to be used in the form of an array
    n -> share of training dataset [0,1]
    d -> d order of the model, set to None if auto_arima is to optimize its value
    """
    
    # Split the dataset into training and testing dataset
    tr_size = int(n*len(data))
    train, test = pm.model_selection.train_test_split(data, train_size = tr_size)
                  
    # Fit the model
    model = pm.auto_arima(train, trace = True, 
                          seasonal = False, stepwise = True, 
                          maxiter = 10, d = d)
    
    # Perform the forecasts
    frc_values = []
    for k in range(len(test)):
        m = model.predict(1)[0]
        frc_values.append(m)
        model.update(test[k])

    # Calculate error metrics
    residuals = model.resid()[len(data)-tr_size+1:len(data)]
    rmse_AR = sqrt(mean_squared_error(test, frc_values))
    MAE_AR = mean_absolute_error(test, frc_values)
    print("The model gives an RMSE of", float("{:.3f}".format(rmse_AR)))
    print("The model gives an MAE of", float("{:.3f}".format(MAE_AR)))

    # Do further analysis
    pm.plot_acf(residuals, lags=20, title = "ACF of residuals", alpha = 0.05)
        
    plt.figure()
    plt.plot(range(0,len(train)), train)
    plt.plot(range(len(train)+1,len(data)+1), test)
    plt.plot(range(len(train)+1,len(data)+1), frc_values)
    plt.xlim([len(train)-40,len(data)])
    plt.xlabel("t")
    plt.ylabel("x")
    plt.legend(["Historical data", "Real values", "Forecasted values"])
    plt.tight_layout()
    plt.show()

In [ ]:
# Call the function
Forecaster(data, 0.8, d = None)